In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
#visualisation libraries
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use(style="seaborn")

import datetime
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import folium
from folium import plugins
from sklearn.model_selection import TimeSeriesSplit
import warnings
warnings.simplefilter("ignore")
from prophet import Prophet
from statsmodels.graphics.tsaplots import plot_acf

In [2]:
df_Hungary= pd.read_csv("df_Hungary.csv" ,encoding='unicode_escape')
df_Hungary

,country,date,population,urbanPop%,#dailyTests,#testsTotal,#dailyTestsPerThousand,#testsTotalPerThousand,#tests7daySmooth,#tests7daySmoothPerThousand,...,E4InternationalSupport,H1PublicInformationCampaigns,H2TestingPolicy,H3ContactTracing,H4EmergencyInvestmentInHealthcare,H5InvestmentInVaccines,H6FacialCoverings,H7VaccinationPolicy,H8ProtectionOfElderlyPeople,mortality
0,Hungary,2020-01-01,9660351,72%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,Hungary,2020-01-02,9660351,72%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,Hungary,2020-01-03,9660351,72%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,Hungary,2020-01-04,9660351,72%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,Hungary,2020-01-05,9660351,72%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
758,Hungary,2021-09-24,9660351,72%,14435.0,6407687.0,1.498,665.101,12793.0,1.328,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.036766
759,Hungary,2021-09-25,9660351,72%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.036766
760,Hungary,2021-09-26,9660351,72%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.036766
761,Hungary,2021-09-27,9660351,72%,0.0,0.0,0.000,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.036737


In [3]:
#shift 21days mortality
df_Hungary_mortality=df_Hungary["mortality"]
df_Hungary_mortality.drop([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21])

22     0.000000
23     0.000000
24     0.000000
25     0.000000
26     0.000000
         ...   
758    0.036766
759    0.036766
760    0.036766
761    0.036737
762    0.000000
Name: mortality, Length: 741, dtype: float64

In [4]:
#删除后21天
df_Hungary.drop(df_Hungary.index[741:762], inplace=True)
df_Hungary

,country,date,population,urbanPop%,#dailyTests,#testsTotal,#dailyTestsPerThousand,#testsTotalPerThousand,#tests7daySmooth,#tests7daySmoothPerThousand,...,E4InternationalSupport,H1PublicInformationCampaigns,H2TestingPolicy,H3ContactTracing,H4EmergencyInvestmentInHealthcare,H5InvestmentInVaccines,H6FacialCoverings,H7VaccinationPolicy,H8ProtectionOfElderlyPeople,mortality
0,Hungary,2020-01-01,9660351,72%,0.0,0.0,0.00,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,Hungary,2020-01-02,9660351,72%,0.0,0.0,0.00,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,Hungary,2020-01-03,9660351,72%,0.0,0.0,0.00,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,Hungary,2020-01-04,9660351,72%,0.0,0.0,0.00,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,Hungary,2020-01-05,9660351,72%,0.0,0.0,0.00,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
737,Hungary,2021-09-03,9660351,72%,10882.0,6155215.0,1.13,638.895,9144.0,0.949,...,0.0,2.0,3.0,1.0,0.0,0.0,2.0,5.0,1.0,0.036974
738,Hungary,2021-09-04,9660351,72%,0.0,0.0,0.00,0.000,9200.0,0.955,...,0.0,2.0,3.0,1.0,0.0,0.0,2.0,5.0,1.0,0.036974
739,Hungary,2021-09-05,9660351,72%,0.0,0.0,0.00,0.000,9256.0,0.961,...,0.0,2.0,3.0,1.0,0.0,0.0,2.0,5.0,1.0,0.036974
740,Hungary,2021-09-06,9660351,72%,28038.0,6183253.0,2.91,641.805,9312.0,0.967,...,0.0,2.0,3.0,1.0,0.0,0.0,2.0,5.0,1.0,0.036955


In [5]:
df_Hungary1=df_Hungary.join(df_Hungary_mortality,how="left",rsuffix="after21days")
df_Hungary1

,country,date,population,urbanPop%,#dailyTests,#testsTotal,#dailyTestsPerThousand,#testsTotalPerThousand,#tests7daySmooth,#tests7daySmoothPerThousand,...,H1PublicInformationCampaigns,H2TestingPolicy,H3ContactTracing,H4EmergencyInvestmentInHealthcare,H5InvestmentInVaccines,H6FacialCoverings,H7VaccinationPolicy,H8ProtectionOfElderlyPeople,mortality,mortalityafter21days
0,Hungary,2020-01-01,9660351,72%,0.0,0.0,0.00,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
1,Hungary,2020-01-02,9660351,72%,0.0,0.0,0.00,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2,Hungary,2020-01-03,9660351,72%,0.0,0.0,0.00,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
3,Hungary,2020-01-04,9660351,72%,0.0,0.0,0.00,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
4,Hungary,2020-01-05,9660351,72%,0.0,0.0,0.00,0.000,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
737,Hungary,2021-09-03,9660351,72%,10882.0,6155215.0,1.13,638.895,9144.0,0.949,...,2.0,3.0,1.0,0.0,0.0,2.0,5.0,1.0,0.036974,0.036974
738,Hungary,2021-09-04,9660351,72%,0.0,0.0,0.00,0.000,9200.0,0.955,...,2.0,3.0,1.0,0.0,0.0,2.0,5.0,1.0,0.036974,0.036974
739,Hungary,2021-09-05,9660351,72%,0.0,0.0,0.00,0.000,9256.0,0.961,...,2.0,3.0,1.0,0.0,0.0,2.0,5.0,1.0,0.036974,0.036974
740,Hungary,2021-09-06,9660351,72%,28038.0,6183253.0,2.91,641.805,9312.0,0.967,...,2.0,3.0,1.0,0.0,0.0,2.0,5.0,1.0,0.036955,0.036955


In [6]:
df_Hungary1.to_csv("df_Hungary_after21days",index=False)